In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import re
import time
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
def remove_special_characters(word):
    return re.sub(r'[^A-Za-z0-9 ]+', '', word)

In [4]:
def get_spotify_instance():
    sp_oauth = SpotifyOAuth(
        client_id=os.getenv('SPOTIPY_CLIENT_ID'),
        client_secret=os.getenv('SPOTIPY_CLIENT_SECRET'),
        redirect_uri=os.getenv('SPOTIPY_REDIRECT_URI'),
        scope='playlist-modify-public'
    )
    
    token_info = sp_oauth.get_cached_token()
    if not token_info:
        token_info = sp_oauth.get_access_token()
    
    return spotipy.Spotify(auth=token_info['access_token'])

In [5]:
def safe_spotify_search(sp, query, retries=3, delay=1, max_results=200):
    results = []
    limit = 50
    offset = 0

    while offset < max_results:
        for attempt in range(retries):
            try:
                response = sp.search(q=f'track:{query}', type='track', limit=limit, offset=offset)
                tracks = response['tracks']['items']
                if not tracks:
                    print(f"No more tracks found after {len(results)} results.")
                    return results
                results.extend(tracks)
                offset += limit
                break  # Exit retry loop if successful
            except spotipy.SpotifyException as e:
                print(f"Spotify API error: {e}")
                if '401' in str(e):
                    print("Token might be invalid or expired. Refreshing token.")
                    sp = get_spotify_instance()
                time.sleep(delay * (2 ** attempt))  # Exponential backoff
        else:
            print("Failed to get a valid response from Spotify after multiple attempts.")
            return results  
    
    return results

In [9]:
def create_playlist_from_sentence():
    # Prompt the user for input
    sentence = input("Please enter the sentence to create a playlist from: ")
    playlist_name = input("Please enter the playlist name: ")

    sp = get_spotify_instance()
    words = [remove_special_characters(word) for word in sentence.split()]
    best_track_ids = []
    fewest_unmatched = len(words)  # Start with the maximum possible unmatched words

    # Try different max_join values to identify the best list
    for max_join in range(6, 1, -1):
        track_ids = []
        i = 0
        unmatched_count = 0
        while i < len(words):
            found_match = False
            for j in range(max_join, 0, -1):
                if i + j <= len(words):
                    combined_word = ' '.join(words[i:i + j])
                    print(f"Searching for: {combined_word}")
                    results = safe_spotify_search(sp, combined_word, max_results=400)
                    if results:
                        for track in results:
                            track_name_cleaned = track['name'].strip().lower()
                            combined_word_cleaned = combined_word.lower()
                            if track_name_cleaned == combined_word_cleaned:
                                track_ids.append(track['id'])
                                found_match = True
                                i += j - 1
                                break
                        if found_match:
                            break
                    time.sleep(0.5)
            if not found_match:
                print(f"No exact match found for words: {' '.join(words[i:i + j])}")
                track_ids.append('69toZvLG490IrN6YtmN4wH')  # Placeholder track ID for track: '?'
                unmatched_count += 1 
            i += 1
        
        # Checking if this configuration has the fewest unmatched words
        if unmatched_count < fewest_unmatched:
            fewest_unmatched = unmatched_count
            best_track_ids = track_ids

    # Creating the playilist using the best configuration
    try:
        user_id = sp.current_user()['id']
        playlist = sp.user_playlist_create(user=user_id, name=playlist_name, public=True)
        if best_track_ids:
            sp.user_playlist_add_tracks(user=user_id, playlist_id=playlist['id'], tracks=best_track_ids)
            print(f"Playlist '{playlist_name}' created successfully!")
        else:
            print("No tracks were added to the playlist.")
    except spotipy.SpotifyException as e:
        print(f"Error creating playlist: {e}")

In [10]:
create_playlist_from_sentence()

Please enter the sentence to create a playlist from: Dude I own this NFT. Do you really think you can get away with theft when you're showing what you stole directly to my face. My lawyers will make an easy job of this case. Prepare to say goodbye to your luscious life and start preparing for the streets. I will ruin you.
Please enter the playlist name: NFT
Searching for: Dude I own this NFT Do
No more tracks found after 0 results.
Searching for: Dude I own this NFT
No more tracks found after 0 results.
Searching for: Dude I own this
No more tracks found after 0 results.
Searching for: Dude I own
No more tracks found after 6 results.
Searching for: Dude I
Searching for: Dude
Searching for: I own this NFT Do you
No more tracks found after 0 results.
Searching for: I own this NFT Do
No more tracks found after 0 results.
Searching for: I own this NFT
No more tracks found after 0 results.
Searching for: I own this
Searching for: NFT Do you really think you
No more tracks found after 0 resu

Searching for: to your luscious life
No more tracks found after 0 results.
Searching for: to your luscious
No more tracks found after 5 results.
Searching for: to your
Searching for: to
No exact match found for words: to
Searching for: your luscious life and start preparing
No more tracks found after 0 results.
Searching for: your luscious life and start
No more tracks found after 0 results.
Searching for: your luscious life and
No more tracks found after 0 results.
Searching for: your luscious life
No more tracks found after 1 results.
Searching for: your luscious
No more tracks found after 49 results.
Searching for: your
No exact match found for words: your
Searching for: luscious life and start preparing for
No more tracks found after 0 results.
Searching for: luscious life and start preparing
No more tracks found after 0 results.
Searching for: luscious life and start
No more tracks found after 0 results.
Searching for: luscious life and
No more tracks found after 12 results.
Searc

No exact match found for words: to
Searching for: your luscious life and start
No more tracks found after 0 results.
Searching for: your luscious life and
No more tracks found after 0 results.
Searching for: your luscious life
No more tracks found after 1 results.
Searching for: your luscious
No more tracks found after 49 results.
Searching for: your
No exact match found for words: your
Searching for: luscious life and start preparing
No more tracks found after 0 results.
Searching for: luscious life and start
No more tracks found after 0 results.
Searching for: luscious life and
No more tracks found after 12 results.
Searching for: luscious life
No more tracks found after 231 results.
Searching for: and start preparing for the
No more tracks found after 1 results.
Searching for: and start preparing for
No more tracks found after 2 results.
Searching for: and start preparing
No more tracks found after 3 results.
Searching for: and start
Searching for: and
No exact match found for words

Searching for: My
No exact match found for words: My
Searching for: lawyers will make
No more tracks found after 0 results.
Searching for: lawyers will
No more tracks found after 5 results.
Searching for: lawyers
Searching for: will make an
Searching for: will make
Searching for: will
Searching for: make an easy
Searching for: make an
Searching for: make
No exact match found for words: make
Searching for: an easy job
No more tracks found after 39 results.
Searching for: of this case
No more tracks found after 101 results.
Searching for: of this
Searching for: of
No exact match found for words: of
Searching for: this case Prepare
No more tracks found after 0 results.
Searching for: this case
Searching for: Prepare to say
No more tracks found after 5 results.
Searching for: Prepare to
Searching for: Prepare
Searching for: to say goodbye
Searching for: to your luscious
No more tracks found after 5 results.
Searching for: to your
Searching for: to
No exact match found for words: to
Searchi

True